### Pranav Srinivas Venkatesh- 1678255
### Thejas Thirthalingaiah- 1678000
### Bharath Kumar Nagaraju- 1604533

Shakespeare sonnet

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Load and preprocess data
def load_shakespeare_dataset():
    # URL where the Shakespeare dataset is hosted
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'

    # Download the dataset
    path_to_file = tf.keras.utils.get_file('shakespeare.txt', url)

    # Read the dataset into memory
    text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

    # Return the loaded dataset
    return text

# Load the dataset
text = load_shakespeare_dataset()
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# Prepare dataset
max_length = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - max_length, step):
    sentences.append(text[i: i + max_length])
    next_chars.append(text[i + max_length])
X = np.zeros((len(sentences), max_length, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_int[char]] = 1
    y[i, char_to_int[next_chars[i]]] = 1

In [ ]:
# Build the model
model = Sequential()
model.add(LSTM(128, input_shape=(max_length, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train the model
model.fit(X, y, batch_size=128, epochs=10)


Epoch 1/10
2905/2905 [==============================] - 339s 116ms/step - loss: 2.3889
Epoch 2/10
2905/2905 [==============================] - 337s 116ms/step - loss: 1.9783
Epoch 3/10
2905/2905 [==============================] - 338s 116ms/step - loss: 1.8381
Epoch 4/10
2905/2905 [==============================] - 337s 116ms/step - loss: 1.7538
Epoch 5/10
2905/2905 [==============================] - 339s 117ms/step - loss: 1.6932
Epoch 6/10
2905/2905 [==============================] - 336s 116ms/step - loss: 1.6462
Epoch 7/10
2905/2905 [==============================] - 336s 116ms/step - loss: 1.6082
Epoch 8/10
2905/2905 [==============================] - 338s 116ms/step - loss: 1.5775
Epoch 9/10
2905/2905 [==============================] - 335s 115ms/step - loss: 1.5515
Epoch 10/10
2905/2905 [==============================] - 337s 116ms/step - loss: 1.5292


In [ ]:
import numpy as np

def sample(preds, temperature=1.0):
    """
    Sample an index from a probability array.

    preds: Numpy array of character predictions.
    temperature: Controls the randomness of predictions.
                 Higher temperature -> more random. Lower -> more deterministic.
    """
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-7) / temperature  # Avoid division by zero; add small value
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [ ]:
# Function to generate text
def generate_text(length, seed):
    generated = ''
    seed = seed[-max_length:]  # Take only the last max_length characters
    generated += seed
    for i in range(length):
        x_pred = np.zeros((1, max_length, len(chars)))
        for t, char in enumerate(seed):
            x_pred[0, t, char_to_int[char]] = 1

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_to_char[next_index]

        generated += next_char
        seed = seed[1:] + next_char  # Shift the seed and add the new char

    return generated


# Generate text
print(generate_text(400, "To be, or not to be: that is the question:"))

 be, or not to be: that is the question:
This will for the comparest Tyerengeruse.

INGlER:
Are, i' kill for my precaiseteding
Nor bride your causun, I devenges to you,
Upon the kings?

Auflel:
Agasion out my partiman, this Remarst,
Thur, hither sand yet un elt his suppero.

hestneinsto's willowh our treat him: and that's fault
The brother, himself, now, a hoops that I consity
With her hee of your honour. Lether, sheew be monale
them, I


Baby Name Generation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, GRU
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import os

In [ ]:

# Path to the folder containing your text files
folder_path = '/content/drive/My Drive/Names'

# List all files in the folder and filter for text files
text_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.txt')]


In [ ]:
file_contents = []  # List to store the contents of each file

for file_name in text_files:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        file_contents.append(content)

In [ ]:
all_text = ' '.join(file_contents).lower()

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
# Tokenize characters
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts([all_text])

# Convert text to sequences of integers
sequences = tokenizer.texts_to_sequences([all_text])[0]

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Create sequences and labels
seq_length = 10  # Length of input sequences
sequences = [sequences[i:i + seq_length + 1] for i in range(len(sequences) - seq_length)]
sequences = np.array(pad_sequences(sequences, maxlen=seq_length + 1, padding='pre'))

# Prepare input and output
X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
X

array([[ 2, 16,  2, ...,  1,  2, 16],
       [16,  2, 18, ...,  2, 16,  2],
       [ 2, 18,  2, ..., 16,  2, 18],
       ...,
       [ 8,  6, 21, ..., 10, 16,  8],
       [ 6, 21,  2, ..., 16,  8,  6],
       [21,  2,  7, ...,  8,  6,  5]], dtype=int32)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

# Build the model
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_length),
    LSTM(100, return_sequences=True),
    LSTM(100),
    Dense(100, activation='relu'),
    Dense(vocab_size, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# EarlyStopping callback
early_stopping = EarlyStopping(monitor='loss', patience=5, verbose=1)

# Train the model with EarlyStopping
model.fit(X, y, batch_size=256, epochs=100, callbacks=[early_stopping])

Epoch 1/100
229/229 [==============================] - 32s 111ms/step - loss: 2.8679 - accuracy: 0.1462
Epoch 2/100
229/229 [==============================] - 25s 108ms/step - loss: 2.4983 - accuracy: 0.2573
Epoch 3/100
229/229 [==============================] - 27s 116ms/step - loss: 2.0746 - accuracy: 0.3839
Epoch 4/100
229/229 [==============================] - 23s 102ms/step - loss: 1.8391 - accuracy: 0.4653
Epoch 5/100
229/229 [==============================] - 25s 108ms/step - loss: 1.7086 - accuracy: 0.5069
Epoch 6/100
229/229 [==============================] - 25s 109ms/step - loss: 1.6155 - accuracy: 0.5361
Epoch 7/100
229/229 [==============================] - 25s 109ms/step - loss: 1.5392 - accuracy: 0.5596
Epoch 8/100
229/229 [==============================] - 23s 101ms/step - loss: 1.4766 - accuracy: 0.5766
Epoch 9/100
229/229 [==============================] - 25s 109ms/step - loss: 1.4189 - accuracy: 0.5922
Epoch 10/100
229/229 [==============================] - 25s 108m

In [ ]:
def generate_name(model, tokenizer, seed_text="A", max_len=20):
    generated_name = seed_text
    for _ in range(max_len):
        # Preprocess the seed_text
        encoded = tokenizer.texts_to_sequences([generated_name])[-1]
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre', truncating='pre')

        # Predict the next character
        prediction = model.predict(encoded, verbose=0).flatten()
        next_index = np.argmax(prediction)
        next_char = tokenizer.index_word.get(next_index, '')

        # Break if the model fails to predict a new character
        if next_char == '':
            break

        generated_name += next_char

    return generated_name

# Example usage
print(generate_name(model, tokenizer, seed_text="Tale", max_len=10))


Talesse
gatoli


Latex Code Generator


In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Replace 'path_to_your_file.txt' with the actual path to your LaTeX file in Google Drive
folder_path = '/content/drive/My Drive/Names/Latex'


In [ ]:
# List all files in the folder and filter for text files
text_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.txt')]


In [ ]:
text_files

['/content/drive/My Drive/Names/Latex/file.txt']

In [ ]:
data = []  # List to store the contents of each file

for file_name in text_files:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        data.append(content)

In [ ]:
#data

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
# Verify dataset is loaded correctly
print("Dataset length:", len(data))
print("First 100 characters of dataset:", data[:100])


Dataset length: 1
First 100 characters of dataset: ['\\documentclass{article}\n\\usepackage{graphicx}\n\\usepackage{amsmath}\n\\usepackage{hyperref}\n\n\\title{Sample LaTeX Document}\n\\author{Your Name}\n\\date{\\today}\n\n\\begin{document}\n\n\\maketitle\n\n\\section{Introduction}\nThis is a sample LaTeX document demonstrating various features.\n\n\\section{Equations}\nHere\'s a simple equation:\n\\begin{equation}\n    E = mc^2\n\\end{equation}\nAnd here\'s another one:\n\\begin{equation}\n    F = ma\n\\end{equation}\n\n\\section{Lists}\n\\subsection{Bullet Points}\n\\begin{itemize}\n    \\item Item 1\n    \\item Item 2\n    \\item Item 3\n\\end{itemize}\n\n\\subsection{Numbered List}\n\\begin{enumerate}\n    \\item First item\n    \\item Second item\n    \\item Third item\n\\end{enumerate}\n\n\\section{Figures}\n\\begin{figure}[h]\n    \\centering\n    \\includegraphics[width=0.5\\textwidth]{example-image-a}\n    \\caption{Example Figure}\n    \\label{fig:example}\n\\end{figure}\n\n

In [ ]:
# If `data` is a list of strings from multiple files, concatenate them into one string
# If it's already a single string in a list (as your output suggests), extract it
latex_data = ''.join(data)  # Use this if `data` is a list of multiple LaTeX document strings
# Or, if `data` is already known to contain exactly one huge string in a list
latex_data = data[0]  # Directly extracting the first element

# Verify the content and length
print(f"First 100 characters of LaTeX data: {latex_data[:100]}")
print(f"Total length of LaTeX data: {len(latex_data)}")


First 100 characters of LaTeX data: \documentclass{article}
\usepackage{graphicx}
\usepackage{amsmath}
\usepackage{hyperref}

\title{Sam
Total length of LaTeX data: 19362


In [ ]:

# Preprocess the data as before (vectorizing characters, creating sequences)
chars = sorted(list(set(latex_data)))
char_to_int = {c: i for i, c in enumerate(chars)}
int_to_char = {i: c for i, c in enumerate(chars)}
seq_length = 40
dataX = []
dataY = []
for i in range(0, len(latex_data) - seq_length, 1):
    seq_in = latex_data[i:i + seq_length]
    seq_out = latex_data[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])


print(f"Length of dataY: {len(dataY)}")
if len(dataY) > 0:
    print(f"Sample from dataY: {dataY[:5]}")
else:
    print("dataY is empty.")


n_patterns = len(dataX)
X = np.reshape(dataX, (n_patterns, seq_length, 1)) / float(len(chars))
y = to_categorical(dataY)

# Define, compile, and train the RNN model as before
# (Refer to the previous model definition and training steps)

# After training, use the model to generate LaTeX code


Length of dataY: 19322
Sample from dataY: [67, 68, 62, 83, 88]


In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense

# Define the RNN model
model = Sequential([
    LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
    Dropout(0.2),
    LSTM(128),
    Dropout(0.2),
    # Final output layer
    Dense(y.shape[1], activation='softmax'),  # Keep this layer unchanged
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Callbacks
#from keras.callbacks import EarlyStopping, ReduceLROnPlateau
#early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, mode='min')

# Train the model with early stopping and learning rate reduction
#model.fit(X, y, epochs=100, batch_size=256, callbacks=[early_stopping, reduce_lr], validation_split=0.2)
model.fit(X, y, epochs=100, batch_size=256,  validation_split=0.2)


Epoch 1/100
61/61 [==============================] - 41s 586ms/step - loss: 3.6518 - val_loss: 3.4761
Epoch 2/100
61/61 [==============================] - 28s 463ms/step - loss: 3.4888 - val_loss: 3.4648
Epoch 3/100
61/61 [==============================] - 32s 528ms/step - loss: 3.4803 - val_loss: 3.4680
Epoch 4/100
61/61 [==============================] - 28s 468ms/step - loss: 3.4732 - val_loss: 3.4642
Epoch 5/100
61/61 [==============================] - 29s 473ms/step - loss: 3.4573 - val_loss: 3.4719
Epoch 6/100
61/61 [==============================] - 28s 467ms/step - loss: 3.4481 - val_loss: 3.4274
Epoch 7/100
61/61 [==============================] - 29s 470ms/step - loss: 3.4115 - val_loss: 3.4017
Epoch 8/100
61/61 [==============================] - 28s 467ms/step - loss: 3.3726 - val_loss: 3.3489
Epoch 9/100
61/61 [==============================] - 28s 465ms/step - loss: 3.3355 - val_loss: 3.3039
Epoch 10/100
61/61 [==============================] - 28s 464ms/step - loss: 3.292

In [ ]:
def adjust_input_sequence(seq, target_length=40):
    # Pad the sequence if it's shorter than the target length
    padding = [0] * (target_length - len(seq))
    padded_seq = seq + padding
    return padded_seq

# Example usage
input_seq = [char_to_int[char] for char in "your_input_sequence_here"]
adjusted_seq = adjust_input_sequence(input_seq)
adjusted_seq = np.reshape(adjusted_seq, (1, 40, 1))  # Reshaping to match the input shape expected by the model

# Now using adjusted_seq for prediction
prediction = model.predict(adjusted_seq)


1/1 [==============================] - 1s 803ms/step


In [ ]:
def generate_latex(model, char_to_int, int_to_char, seed_text, sequence_length, num_chars_to_generate=400):
    """
    Generate LaTeX code using a trained RNN model.

    Parameters:
    - model: The trained RNN model.
    - char_to_int: Dictionary mapping characters to integers.
    - int_to_char: Dictionary mapping integers back to characters.
    - seed_text: A seed sequence to start generating from.
    - sequence_length: The length of sequences the model was trained with.
    - num_chars_to_generate: Number of characters to generate.

    Returns:
    - generated_text: The generated LaTeX code as a string.
    """
    generated_text = ""
    # Adjust seed text to match the expected sequence length
    if len(seed_text) > sequence_length:
        seed_text = seed_text[-sequence_length:]
    elif len(seed_text) < sequence_length:
        # Pad seed_text if it's shorter than required
        seed_text = seed_text.rjust(sequence_length)

    for _ in range(num_chars_to_generate):
        # Convert the seed text to a sequence of integers
        input_seq = [char_to_int.get(char, 0) for char in seed_text]  # Use .get() to handle unseen chars
        input_seq = np.reshape(input_seq, (1, sequence_length, 1))
        input_seq = input_seq / float(len(char_to_int))

        # Predict the next character
        prediction = model.predict(input_seq, verbose=0)
        index = np.argmax(prediction)
        next_char = int_to_char[index]

        generated_text += next_char
        seed_text = seed_text[1:] + next_char  # Update seed text

    return generated_text

# Adjust the sequence_length accordingly
sequence_length = 40  # This should match the training sequence length
seed_text = "\\begin{document}\n"  # Your initial seed text
# Make sure to pass the sequence_length to the function
generated_latex = generate_latex(model, char_to_int, int_to_char, seed_text, sequence_length, num_chars_to_generate=400)
print(generated_latex)


 \section{Sigure}
\uebtections:
\regr \refeent daathres.oure}atio 
\eec donueens siuese to inpr piis aiatr.

\eeccion{Lusereocoio}
Thid{fofo}
\erg{ettmaat}

\section{cent}
\dogsettillss{articele}
\usepcckage{amssath}
\usepackage{lissing}
\usepackage{tippri}
\usepackage{lissiigs}
\usepackage{tiopat}
\usepackage{tiopr
\usepackage{aissrcn}
\usepackage{tiksiin}
\usepackage{tiopri}
\usepackage{lissiigs


In [ ]:
print(generated_latex)

In [ ]:
# Create a dummy input sequence that matches the model's expected input shape
dummy_input = np.zeros((1, 40, 1))  # Assuming 40 is the sequence length your model was trained on

# Try making a prediction with the dummy input
try:
    dummy_prediction = model.predict(dummy_input)
    print("Model prediction successful.")
except Exception as e:
    print(f"Model prediction failed: {e}")


1/1 [==============================] - 0s 36ms/step
Model prediction successful.


In [ ]:
def generate_latex(model, char_to_int, int_to_char, seed_text, sequence_length, num_chars_to_generate=400):
    """
    Generate LaTeX code using a trained RNN model.

    Parameters:
    - model: The trained RNN model.
    - char_to_int: Dictionary mapping characters to integers.
    - int_to_char: Dictionary mapping integers back to characters.
    - seed_text: A seed sequence to start generating from.
    - sequence_length: The length of sequences the model was trained with.
    - num_chars_to_generate: Number of characters to generate.

    Returns:
    - generated_text: The generated LaTeX code as a string.
    """
    generated_text = ""

    # Adjust the seed text to the sequence length expected by the model
    seed_text = seed_text[-sequence_length:]  # Use the last 'sequence_length' characters
    if len(seed_text) < sequence_length:
        # Pad the seed_text if shorter than expected
        padding = ' ' * (sequence_length - len(seed_text))  # Assuming space can be used for padding
        seed_text = padding + seed_text

    for _ in range(num_chars_to_generate):
        # Convert the seed text to a sequence of integers
        input_seq = [char_to_int.get(char, 0) for char in seed_text]  # Use .get() to handle unseen chars
        input_seq = np.reshape(input_seq, (1, sequence_length, 1))
        input_seq = input_seq / float(len(char_to_int))

        # Predict the next character
        prediction = model.predict(input_seq, verbose=0)
        index = np.argmax(prediction)
        next_char = int_to_char[index]

        generated_text += next_char
        seed_text = seed_text[1:] + next_char  # Update seed text with the next character

    return generated_text

# Example usage



In [ ]:
seed_text = "\\begin{document}\n"  # Adjust your seed text as needed
sequence_length = 40  # This should match the sequence length your model expects
generated_latex = generate_latex(model, char_to_int, int_to_char, seed_text, sequence_length, num_chars_to_generate=400)
print(generated_latex)

 \section{Sigure}
\uebtections:
\regr \refeent daathres.oure}atio 
\eec donueens siuese to inpr piis aiatr.

\eeccion{Lusereocoio}
Thid{fofo}
\erg{ettmaat}

\section{cent}
\dogsettillss{articele}
\usepcckage{amssath}
\usepackage{lissing}
\usepackage{tippri}
\usepackage{lissiigs}
\usepackage{tiopat}
\usepackage{tiopr
\usepackage{aissrcn}
\usepackage{tiksiin}
\usepackage{tiopri}
\usepackage{lissiigs
